# import package

In [1]:
import pandas as pd 
import numpy as np
import warnings
from functools import partial
import re


In [2]:
def get_int(x):
    """
    permet de recuperer les chiffre dans une cellule, si c'est une chaine de caratere, retourn np.nan
    """
    try : 
        x = int(x)
    except: 
        return np.nan
    return x

def supprimer_caracteres_speciaux(chaine):
    """
    permet de supprimer tous les caractere speciaux dans une chaine de caractere
    """
    chaine_propre =  ''
    for caractere in chaine:
        if caractere.isalnum() or caractere.isspace() or caractere in ['-', ';','/', '\\', '|']:
            chaine_propre += caractere
    return chaine_propre

# Ignorer tous les warnings de pandas
warnings.simplefilter(action='ignore', category=Warning)

# read file 

In [3]:
# lecture du fichier principale
df = pd.read_csv('data.csv', sep = ';', encoding='utf8')
df.head()

,nom_du_festival,envergure_territoriale,region_principale_de_deroulement,departement_principal_de_deroulement,commune_principale_de_deroulement,code_postal_de_la_commune_principale_de_deroulement,code_insee_commune,code_insee_epci_collage_en_valeur,libelle_epci_collage_en_valeur,numero_de_voie,...,sous_categorie_musique,sous_categorie_musique_cnm,sous_categorie_cinema_et_audiovisuel,sous_categorie_arts_visuels_et_arts_numeriques,sous_categorie_livre_et_litterature,periode_principale_de_deroulement_du_festival,identifiant_agence_a,identifiant,geocodage_xy,identifiant_cnm
0,Avoine Zone Groove,Intercommunale,Centre-Val de Loire,Indre-et-Loire,Avoine,37420,37011,200043081,"CC Chinon, Vienne et Loire",NaN,...,"Jazz, Blues",07- Musiques actuelles sans distinction,NaN,NaN,NaN,Saison (21 juin - 5 septembre),NaN,FEST_37011_550,"47.2237767737, 0.18754463005",333.0
1,Festival des correspondances des arts,Départementale,Hauts-de-France,Aisne,Braine,2220,2110,240200501,CC du Val de l'Aisne,30,...,NaN,NaN,NaN,NaN,NaN,Après-saison (6 septembre - 31 décembre),NaN,FEST_02110_1021,"49.3484719318, 3.53596629744",NaN
2,Soirées d'automne - festival itinérant,NaN,Provence-Alpes-Côte d'Azur,Vaucluse,Carpentras,84203,84031,248400053,CA Ventoux-Comtat-Venaissin (COVE),NaN,...,NaN,09- Pluridisciplinaire,NaN,NaN,NaN,Après-saison (6 septembre - 31 décembre),NaN,FEST_84031_1284,"44.0593802565, 5.06134844776",675.0
3,Macki Music Festival,NaN,Île-de-France,Yvelines,Carrières-sur-Seine,78420,78124,200058519,Saint-Germain Boucles de Seine,NaN,...,"Musiques du monde, Musiques traditionnelles, R...",02- Musiques amplifiées ou électroniques,NaN,NaN,NaN,Saison (21 juin - 5 septembre),NaN,FEST_78124_1288,"48.9119335064, 2.17838235323",1708.0
4,Play mobile,NaN,Île-de-France,Hauts-de-Seine,Chatillon,92320,92020,200057966,Vallée Sud Grand Paris (T2),NaN,...,NaN,NaN,NaN,NaN,NaN,Avant-saison (1er janvier - 20 juin),NaN,FEST_92020_1579,"48.8034091756, 2.28799131883",NaN


# EPCI

In [4]:
# recuperation ce code_epci et nom_epci, suppression des valeur vide, ssuppression des valeur dupliquer, remise a 0 des index,
# enregistrement dans un txt le resultat obtenu
epci = df.iloc[:, [7, 8]]
epci['code_insee_epci_collage_en_valeur'] = epci['code_insee_epci_collage_en_valeur'].map(get_int)
epci = epci.dropna(axis=0)
epci = epci.drop_duplicates(subset='code_insee_epci_collage_en_valeur')
epci.reset_index(drop=True, inplace=True)
epci['code_insee_epci_collage_en_valeur'] = epci['code_insee_epci_collage_en_valeur'].astype('int')
epci.index = epci.index + 1
epci = pd.concat([pd.DataFrame({'code_insee_epci_collage_en_valeur' : [0], 'libelle_epci_collage_en_valeur' : ['unknow']}), epci])
epci.to_csv('epci.txt', sep='\t', index=False, encoding='utf8', header=False)

# envergure

In [5]:
# recuperation ce envergure teritorial et nom_epci, suppression des valeur vide, ssuppression des valeur dupliquer, reset des index(pour utiliser comme id),
# enregistrement dans un txt le resultat obtenu
envergure = df['envergure_territoriale'].to_frame()
envergure['envergure_territoriale'] = envergure['envergure_territoriale'].str.lower()
envergure.dropna(inplace = True)
envergure.drop_duplicates(inplace = True)
envergure = envergure.reset_index(drop=True)
envergure.index = envergure.index + 1

# ajout de unknow pour les festival qui n'ont pas de envergure, resoudre les erreur de cle etrangere
envergure = pd.concat([pd.DataFrame({'envergure_territoriale' : ['unknow']}), envergure])
envergure.to_csv('envergure.txt', sep='\t', encoding='utf8', header=False)

# region

In [6]:
# recuperation de region_principale, suppression des valeur vide, ssuppression des valeur dupliquer, reset des index(pour utiliser comme id),
# enregistrement dans un txt le resultat obtenu
region = df['region_principale_de_deroulement']
region.dropna(inplace = True)
region.drop_duplicates(inplace = True)
region.index = region.index + 1

# ajout de unknow pour les festival qui n'ont pas de region, resoudre les erreur de cle etrangere
region = pd.concat([pd.DataFrame({'region_principale_de_deroulement' : ['unknow']}), region])
region.to_csv('region.txt', sep='\t', encoding='utf8', header=False)

# departement

In [7]:
# recuperation de departement et region_principale, suppression des valeur vide, ssuppression des valeur dupliquer, reset des index(pour utiliser comme id),
# enregistrement dans un txt le resultat obtenu
dep = df[['region_principale_de_deroulement', 'departement_principal_de_deroulement']]
dep = dep.drop_duplicates(subset =  'departement_principal_de_deroulement')
region['id_region'] = region.index.map(lambda x: x)

# merge departement et region pour recuperer les cle de region grace a id
dep = pd.merge(dep, region, how='left', left_on='region_principale_de_deroulement', right_on='region_principale_de_deroulement')
dep = dep[['departement_principal_de_deroulement', 'id_region']]
dep.index = dep.index + 1

# ajout de unknow pour resoudre les erreur de cle primaire
dep = pd.concat([pd.DataFrame({'id_region' : [0], 'departement_principal_de_deroulement' : ['inconu']}), dep])
dep = dep[['departement_principal_de_deroulement', 'id_region']]
dep.to_csv('departement.txt', sep='\t', encoding='utf8', header=False)

In [8]:
dep

,departement_principal_de_deroulement,id_region
0,inconu,0
1,Indre-et-Loire,1
2,Aisne,2
3,Vaucluse,3
4,Yvelines,4
...,...,...
107,isère,27
108,Île de ’Ua Pou,504
109,allier,27
110,En ligne,4


# commune

In [9]:
# recuperation des champ pour commune, suppression des valeur vide, ssuppression des valeur dupliquer, reset des index,
# enregistrement dans un txt le resultat obtenu
commune = df[['code_insee_commune', 'code_postal_de_la_commune_principale_de_deroulement','departement_principal_de_deroulement', 'commune_principale_de_deroulement']]
dep['id_departement'] = dep.index.map(lambda x: x)

# merge avec departement pour recuperer les cle grace a id, uppression des doublon et valeur vide
commune = pd.merge(commune, dep, how='left', on='departement_principal_de_deroulement')
commune = commune.drop_duplicates(subset=['code_insee_commune'])
commune['code_insee_commune'] = commune['code_insee_commune'].map(get_int)
commune = commune.dropna(axis=0)
commune['code_insee_commune'] = commune['code_insee_commune'].astype('int')

# code_insee cntient des str, remlacement de c'est str par des int ou suppression
commune['code_postal_de_la_commune_principale_de_deroulement'] = commune['code_postal_de_la_commune_principale_de_deroulement'].map(get_int).fillna(0).astype('int')

# recuperation des champs a enregistrer
commune = commune[['code_insee_commune', 'commune_principale_de_deroulement', 'code_postal_de_la_commune_principale_de_deroulement','id_departement']]

# ajout de unknow pour resoudre les erreur de cle
commune = pd.concat([pd.DataFrame({'code_insee_commune' : [0], 'commune_principale_de_deroulement' : ['unknow'], 'code_postal_de_la_commune_principale_de_deroulement' : [0], 'id_departement' : [0]}), commune])
commune.to_csv('commune.txt', sep='\t', encoding='utf8', header=False, index = False)

In [10]:
commune

,code_insee_commune,commune_principale_de_deroulement,code_postal_de_la_commune_principale_de_deroulement,id_departement
0,0,unknow,0,0
0,37011,Avoine,37420,1
1,2110,Braine,2220,2
2,84031,Carpentras,84203,3
3,78124,Carrières-sur-Seine,78420,4
...,...,...,...,...
7269,3110,Espinasse-Vozelle,3110,17
7271,85094,Foussais-Payré,85240,78
7273,1197,Journans,1250,36
7274,6110,Le Cannet,6100,8


# adresse

In [11]:
# recuperation des champ pour adresse, suppression des valeur vide, ssuppression des valeur dupliquer, reset des index(pour utiliser comme id),
# enregistrement dans un txt le resultat obtenu
adresse = df[['numero_de_voie', 'type_de_voie_rue_avenue_boulevard_etc', 'nom_de_la_voie', 'complement_d_adresse_facultatif', 'code_insee_commune']]
adresse = adresse.fillna('pas')
a = (adresse['numero_de_voie'] != 'pas') & (adresse['type_de_voie_rue_avenue_boulevard_etc'] != 'pas') & (adresse['nom_de_la_voie'] != 'pas')
adresse = adresse[a]
adresse = adresse.reset_index(drop=True)
adresse['code_insee_commune'] = adresse['code_insee_commune'].map(get_int)

# merge avec commune pour obtenir les clé etrangere 
adresse = pd.merge(adresse, commune, how='inner', on='code_insee_commune')
adresse = adresse[['numero_de_voie', 'type_de_voie_rue_avenue_boulevard_etc', 'nom_de_la_voie', 'complement_d_adresse_facultatif', 'code_insee_commune']]
adresse = adresse.dropna(axis=0)
adresse['code_insee_commune'] = adresse['code_insee_commune'].astype('int')
adresse.index = adresse.index + 1

#ajout de unknow pour resoudre le probleme de cle
adresse = pd.concat([pd.DataFrame({'numero_de_voie' : [0], 'type_de_voie_rue_avenue_boulevard_etc' : ['unknow'], 'nom_de_la_voie' : ['unknow'], 'complement_d_adresse_facultatif' : ['unknow'], 'code_insee_commune' : [0]}), adresse])
adresse.to_csv('adresse.txt', sep='\t', encoding='utf8', header=False)

# categorie

In [12]:
# recuperation des champ pour categorie, suppression des valeur vide, ssuppression des valeur dupliquer, reset des index(pour utiliser comme id),
# enregistrement dans un txt le resultat obtenu
categorie = df[['discipline_dominante']]
categorie = categorie['discipline_dominante'].str.split(',', expand=True)
categorie = list(categorie[0]) + list(categorie[1])
categorie = list(set(categorie))
categorie.remove(None)
categorie = pd.DataFrame({'nom_categorie' : categorie})
categorie.index = categorie.index + 1
categorie.to_csv('categorie.txt', sep='\t', encoding='utf8', header=False)

# SOUS CATEGORIE

In [13]:
# création de sous categorie
nom = ['spectacle vivant', 'musique', 'musique cnm', 'cinema et audiovisuel', 'arts visuels et arts numeriques', 'livre et litterature']
sous_categorie = pd.DataFrame({'nom_sous_categorie' : nom})
sous_categorie.index = sous_categorie.index + 1
sous_categorie.to_csv('sous_categorie.txt', sep = '\t', header=False)

In [14]:
sous_categorie

,nom_sous_categorie
1,spectacle vivant
2,musique
3,musique cnm
4,cinema et audiovisuel
5,arts visuels et arts numeriques
6,livre et litterature


# activite et sous_categorie_activite

In [15]:
# recuperation des champ pour activite, suppression des valeur vide, ssuppression des valeur dupliquer, reset des index(pour utiliser comme id),
# enregistrement dans un txt le resultat obtenu
cat = ['sous_categorie_' + i.replace(' ', '_') for i in nom]
activite = pd.DataFrame({'nom_activite' : [], 'sous_categorie' : []})
# boucle sur chaque sous categorie
for i, j in enumerate(cat):
    acti = df[cat[i]]
    #split en plusieur colonne
    acti = acti.str.replace(';', ',').str.split(',', expand=True)
    new = []
    #parcour pour recuperer chause element
    for col in list(acti.columns):
        new = new + list(acti[col])
    #suppression des doublon 
    new = list(set(new))
    try : 
        new.remove(None)
        new.remove('')
    except : 
        pass

    # ajout des activite dans le dataframe
    new = pd.DataFrame(({'nom_activite' : new, 'sous_categorie' : [i]* len(new)}))
    activite = pd.concat([activite, new], ignore_index=True)
    # break

# conversion au bon type des id, supression doublon, enregistrement
activite['sous_categorie'] = activite['sous_categorie'].astype('int')
activite.drop_duplicates(subset='nom_activite', inplace = True)
activite.index = activite.index + 1
activite['nom_activite'].to_frame().to_csv('activite.txt', sep='\t', encoding='utf8', header=False)
activite2 = activite['nom_activite'].to_frame()

# recuperation des sous categorie de chaque  activite
#sous_categorie_activite
activite['id'] = activite.index
activite = activite[['sous_categorie', 'id']]
activite.rename(columns = {'id' : 'id_activite'}, inplace = True)
activite['sous_categorie'] = activite['sous_categorie'] + 1
activite = activite.drop_duplicates()
activite.to_csv('sous_categorie_activite.txt', sep='\t', encoding='utf8', header=False, index = False)

# periode deroulement

In [16]:
# recuperation des champ pour periode deroulment, suppression des valeur vide, ssuppression des valeur dupliquer, reset des index(pour utiliser comme id),
# enregistrement dans un txt le resultat obtenu
df['periode_principale_de_deroulement_du_festival'] = df['periode_principale_de_deroulement_du_festival'].str.lower()
periode = df['periode_principale_de_deroulement_du_festival'].to_frame()
periode = periode.drop_duplicates()
periode = periode.dropna()
periode = periode.reset_index(drop = True)
periode.index = periode.index + 1
periode = pd.concat([pd.DataFrame({'periode_principale_de_deroulement_du_festival' : ['inconu']}), periode])
periode.to_csv('periode.txt', sep = '\t', header = False)

# decenie

In [17]:
# recuperation des champ pour decennie, suppression des valeur vide, ssuppression des valeur dupliquer, reset des index(pour utiliser comme id),
# enregistrement dans un txt le resultat obtenu
decenie = df['decennie_de_creation_du_festival'].to_frame()
decenie = decenie.drop_duplicates(subset = 'decennie_de_creation_du_festival')
decenie = decenie.dropna()
decenie = decenie.reset_index(drop = True)
decenie.index = decenie.index + 1
decenie = pd.concat([pd.DataFrame({'decennie_de_creation_du_festival' : ['inconu']}), decenie])
decenie.to_csv('decennie.txt', sep = '\t', header = False)

# festival

In [18]:
# recuperation des champ pour festival, suppression des valeur vide, ssuppression des valeur dupliquer, reset des index(pour utiliser comme id),
# enregistrement dans un txt le resultat obtenu
col = ['nom_du_festival', 'site_internet_du_festival', 'adresse_e_mail', 'annee_de_creation_du_festival', 'identifiant_agence_a', 'identifiant_cnm']
festival = df[col]
festival[['geocodage_x', 'geocodage_y']] = df['geocodage_xy'].str.split(',', expand=True)
col = col + ['geocodage_x', 'geocodage_y']
festival['identifiant_cnm'] = festival['identifiant_cnm'].fillna(0).astype('int')
festival['annee_de_creation_du_festival'] = festival['annee_de_creation_du_festival'].fillna(0)


# envergure
# merge de festival avec envergure pour recup le id
e = envergure
e['id_envergure'] = e.index
festival.loc[:, ['envergure_territoriale']] = df.loc[:, ['envergure_territoriale']]
festival = pd.merge(festival, e, how='left', left_on='envergure_territoriale', right_on='envergure_territoriale')
festival = festival[col + ['id_envergure']]
festival['id_envergure'] = festival['id_envergure'].fillna(0)
festival['id_envergure'] = festival['id_envergure'].astype('int')

# code_epci
# merge de festival avec epci pour recup le id
e = epci
festival.loc[:, ['code_insee_epci_collage_en_valeur']] = df.loc[:, ['code_insee_epci_collage_en_valeur']].map(get_int).fillna(0).astype('int')
festival = pd.merge(festival, e, how='left', left_on='code_insee_epci_collage_en_valeur', right_on='code_insee_epci_collage_en_valeur')
festival = festival[col + ['id_envergure', 'code_insee_epci_collage_en_valeur']]

# code_insee_commune
# merge de festival avec commune pour recup le id
festival['code_insee_commune'] = df.loc[:, ['code_insee_commune']]

# date deroulement
# merge de festival avec periode pour recup le id
e = periode
e['id_periode'] = e.index
festival['periode_principale_de_deroulement_du_festival'] = df.loc[:, ['periode_principale_de_deroulement_du_festival']]
festival = pd.merge(festival, e, how='left', left_on='periode_principale_de_deroulement_du_festival', right_on='periode_principale_de_deroulement_du_festival')
festival = festival[col + ['id_envergure', 'code_insee_epci_collage_en_valeur', 'code_insee_commune', 'id_periode']]
festival['id_periode'] = festival['id_periode'].fillna(0)
festival['id_periode'] = festival['id_periode'].astype('int')

# decenie creation
# merge de festival avec decenie_creation pour recup le id
e = decenie
e['id_decenie'] = e.index
festival['decennie_de_creation_du_festival'] = df.loc[:, ['decennie_de_creation_du_festival']]
festival = pd.merge(festival, e, how='left', left_on='decennie_de_creation_du_festival', right_on='decennie_de_creation_du_festival')
festival = festival[col + ['id_envergure', 'code_insee_epci_collage_en_valeur', 'id_periode', 'id_decenie', 'code_insee_commune']]
festival['id_decenie'] = festival['id_decenie'].fillna(0)
festival['id_decenie'] = festival['id_decenie'].astype('int')

# code insee commune : deja fait au dessus
# boucle pour corriger les erreur et valeur manquante
festival['code_insee_commune'] = festival['code_insee_commune'].map(get_int).fillna(0).astype('int')
# festival = pd.merge(festival, commune, how = 'inner', on = 'code_insee_commune')
def verif_insee(x, code):
    if x in code:
        return x
    else : 
        return 0

code = list(commune['code_insee_commune'])
n_verif_insee = partial(verif_insee, code=code)
festival['code_insee_commune'] = festival['code_insee_commune'].map(n_verif_insee)
festival.index = festival.index + 1
festival.to_csv('festival.txt', sep = '\t', header = False)

In [19]:
festival

,nom_du_festival,site_internet_du_festival,adresse_e_mail,annee_de_creation_du_festival,identifiant_agence_a,identifiant_cnm,geocodage_x,geocodage_y,id_envergure,code_insee_epci_collage_en_valeur,id_periode,id_decenie,code_insee_commune
1,Avoine Zone Groove,www.avoinezonegroove.fr,NaN,2000,NaN,333,47.2237767737,0.18754463005,0,200043081,1,1,37011
2,Festival des correspondances des arts,https://braine.fr/,correspondance-des-arts@braine.fr,2018,NaN,0,49.3484719318,3.53596629744,0,240200501,2,2,2110
3,Soirées d'automne - festival itinérant,www.soireesdautomne.com,info@passagersduzinc.com,0,NaN,675,44.0593802565,5.06134844776,0,248400053,2,2,84031
4,Macki Music Festival,https://www.mackimusicfestival.fr/,mackimusicfestival@gmail.com,2014,NaN,1708,48.9119335064,2.17838235323,0,200058519,1,2,78124
5,Play mobile,http://www.theatreachatillon.com/play-mobile.php,NaN,01/01/2016 00:00,NaN,0,48.8034091756,2.28799131883,0,200057966,3,2,92020
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7279,Langues de voix,http://musicalsol.fr/nos-concerts/langues-de-v...,programmation@musicalsol.fr,2019,NaN,4669,43.2513741105,2.31315453913,0,200035715,3,2,11279
7280,Festival de la parole et du livre,www.ville-saintlaurentduvar.fr,brigitte.belhassen@saintlaurentduvar.fr,1995,NaN,0,43.6859625772,7.18218791815,0,200030195,2,4,6123
7281,Saint Mandé Classic Jazz Festival,www.saint-mande-festival.com,NaN,01/01/2018 00:00,NaN,3837,48.842440243,2.4192191088,0,200057941,3,2,94067
7282,Charlie jazz festival,www.charlie-jazz.com,com@charlie-jazz.com,1997,NaN,14,43.4497831674,5.26357787665,0,200054807,1,4,13117


# adresse_festival

In [20]:
# creation colonne id pour les deux
adresse['id_adresse'] = adresse.index
festival['id_festival'] = festival.index

# merge de festival avec adresse pour recup le id
ad_fes = pd.merge(adresse, festival, how = 'right', left_on= 'code_insee_commune', right_on='code_insee_commune')

# recuperation des colonne utile
ad_fes = ad_fes[['id_adresse', 'id_festival']]

# remplissage des valeur vide par 0 et suppresion des doublon
ad_fes['id_adresse'] = ad_fes['id_adresse'].fillna(0).astype('int')
ad_fes = ad_fes.drop_duplicates(subset = ['id_festival'])
ad_fes.reset_index(drop = True, inplace = True)
ad_fes.to_csv('adresse_festival.txt', sep = '\t', index = False, header = False)

# categorie_festival

In [21]:
categorie['id_categorie'] = categorie.index
#recuperation des categorie et creation de chaine de caractere pour untiliser expression reguliere
cat = '|'.join(list(categorie['nom_categorie']))
dom = df[['discipline_dominante', 'nom_du_festival']]

#merge pour recup la colonne discipline_dominante  de chaque festival
cat_fes = pd.merge(festival, dom, how = 'left', left_on= 'nom_du_festival', right_on='nom_du_festival')[['id_festival', 'discipline_dominante']]

# initialisation de liste en vu de creer un dataframe avec 
id_f = []
id_cat = []

#iteration sur chque ligne
for index, row in cat_fes.iterrows():
    a = row['discipline_dominante']
    if a != None:
        # split au cas ou c'est plusieurs discipline puis boucle sur chaque valeur et test le regex
        a = a.split(',')
        for i in a:
            if re.search(cat, i):
                id_f.append(row['id_festival'])
                id_cat.append(i)

#creation dataframe avec les categorie de chause festival
cat_fes = pd.DataFrame({'categorie' : id_cat, 'id_festival' : id_f})
cat_fes = pd.merge(cat_fes, categorie, how = 'left', left_on = 'categorie', right_on = 'nom_categorie')

# recup des id et enregistrement
cat_fes = cat_fes[['id_categorie', 'id_festival']]
cat_fes = cat_fes.drop_duplicates(subset = ['id_categorie', 'id_festival'])
cat_fes.to_csv('categorie_festival.txt', sep = '\t', index = False, header = False)

# sous_categorie_festival

In [22]:
sous_categorie['id_sous_categorie'] = sous_categorie.index
#recuperation des categorie et creation de chaine de caractere pour untiliser expression reguliere
nom = ['spectacle vivant', 'musique', 'musique cnm', 'cinema et audiovisuel', 'arts visuels et arts numeriques', 'livre et litterature']
cat = ['sous_categorie_' + i.replace(' ', '_') for i in nom]

id_f = []
sc = []

#iteration sur chque ligne
for index, row in df.iterrows():
    for i in cat:
        if type(row[i]) is str:
            id_f.append(index)
            sc.append(i)
    
sous_categorie['nom_sous_categorie'] = cat

#creation dataframe avec les categorie de chaque festival
sous_cat_fes = pd.DataFrame({'sous_categorie' : sc, 'id_festival' : id_f})
sous_cat_fes['id_festival'] = sous_cat_fes['id_festival'] + 1
sous_cat_fes = pd.merge(sous_cat_fes, sous_categorie, how = 'inner', left_on = 'sous_categorie', right_on = 'nom_sous_categorie')

# recup des id et enregistrement
sous_cat_fes = sous_cat_fes[['id_sous_categorie', 'id_festival']]
sous_cat_fes = sous_cat_fes.drop_duplicates(subset = ['id_sous_categorie', 'id_festival'])
sous_cat_fes.to_csv('sous_categorie_festival.txt', sep = '\t', index = False, header = False)

# activite_festival

In [23]:
acti = '|'.join(list(activite2['nom_activite'].astype('str')))
nom = ['spectacle vivant', 'musique', 'musique cnm', 'cinema et audiovisuel', 'arts visuels et arts numeriques', 'livre et litterature']
cat = ['sous_categorie_' + i.replace(' ', '_') for i in nom]

# initialisation de liste en vu de creer un dataframe avec 
id_f = []
id_a = []

#iteration sur chque ligne
for index, row in df.iterrows():
    for i in cat:
        a = row[i]
        if type(a) is str:
            # split au cas ou c'est plusieurs discipline puis boucle sur chaque valeur et test le regex
            a = a.split('-')
            try :
                float(a[0])
                a.pop(0)
            except:
                pass
            finally:
                a = a[-1].replace(';', ',').split(',')
            for j in a:
                j = supprimer_caracteres_speciaux(j)
                if re.search(j, acti): 
                    id_f.append(index)
                    id_a.append(j)
                    # print(id_f, id_a)
    

#creation dataframe avec les categorie de chause festival
acti_fes = pd.DataFrame({'id_a' : id_a, 'id_festival' : id_f})
# print(acti_fes)

v2 = activite2
v2['id_activite'] = v2.index 
v2['nom_activite'] = v2['nom_activite'].astype('str')#.map(supprimer_caracteres_speciaux)
acti_fes = pd.merge(acti_fes, v2, how = 'inner', left_on = 'id_a', right_on = 'nom_activite')
acti_fes = acti_fes.dropna(axis=0)
acti_fes['id_activite'] = acti_fes['id_activite'].astype('int')
acti_fes.head()

# recup des id et enregistrement
acti_fes = acti_fes[['id_activite', 'id_festival']]
acti_fes = acti_fes.drop_duplicates()
acti_fes['id_festival'] = acti_fes['id_festival'] + 1
acti_fes.to_csv('activite_festival.txt', sep = '\t', index = False, header = False)

# sous_categorie_categorie

In [24]:
# recup sous categorie
nom = ['spectacle vivant', 'musique', 'musique cnm', 'cinema et audiovisuel', 'arts visuels et arts numeriques', 'livre et litterature']
cat = ['sous_categorie_' + i.replace(' ', '_') for i in nom]
sous_cat_cat = df[['discipline_dominante'] + cat]

id_cat = []
id_sous_cat = []

# bouvle sur chaque ligne de df pour recup les sous_cat et cat
for index, row in df.iterrows():
    for i in cat: 
        if type(row[i]) is str:
            dis_dom = row['discipline_dominante'].split(',')
            for j in dis_dom : 
                id_cat.append(j)
                id_sous_cat.append(i)

# creation du dataframe
sous_cat_cat = pd.DataFrame({'sous_categorie' : id_sous_cat, 'categorie' : id_cat})

# merge pour recup les id
sous_cat_cat = pd.merge(sous_cat_cat, sous_categorie, how='left', left_on='sous_categorie', right_on='nom_sous_categorie')
sous_cat_cat = pd.merge(sous_cat_cat, categorie, how='left', left_on='categorie', right_on='nom_categorie')

# reup des champs utile, suppression des valeur vide et des doublon et enregistrement
sous_cat_cat = sous_cat_cat[['id_sous_categorie', 'id_categorie']]
sous_cat_cat = sous_cat_cat.dropna(axis=0).drop_duplicates()
sous_cat_cat.to_csv('sous_categorie_categorie.txt', sep = '\t', index = False, header = False)

In [25]:
sous_cat_cat

,id_sous_categorie,id_categorie
0,2,6
1,3,6
5,1,2
19,4,3
20,4,4
23,5,7
24,5,5
31,2,2
40,1,8
43,1,7
